# Library Heatmaps Explanation
# Explicação de Mapas de Calor da Biblioteca

## Summary / Resumo

This notebook demonstrates how to generate and interpret heatmap visualizations of metasurface parameter spaces.

Este notebook demonstra como gerar e interpretar visualizações de mapas de calor de espaços de parâmetros de metassuperfície.

## Inputs / Entradas

**Required files / Arquivos necessários:**
- Cleaned library CSV or Parquet file with derived columns
- Arquivo CSV ou Parquet de biblioteca limpa com colunas derivadas

**Expected columns / Colunas esperadas:**
- `L_x`, `L_y`: Nanopilar dimensions / Dimensões do nanopilar
- `phase_TE`, `amp_TE`: TE transmission parameters / Parâmetros de transmissão TE
- `phase_TM`, `amp_TM`: TM transmission parameters / Parâmetros de transmissão TM

In [ ]:
# Import required packages / Importar pacotes necessários
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add src to path / Adicionar src ao caminho
repo_root = Path.cwd().parent.parent
sys.path.insert(0, str(repo_root / 'src'))

from meta_library import phase_matching

print('✓ Packages loaded successfully / Pacotes carregados com sucesso')

## Configuration / Configuração

Set paths and parameters for heatmap generation.

Definir caminhos e parâmetros para geração de mapas de calor.

In [ ]:
# Configuration / Configuração
library_file = repo_root / 'library_processed' / 'example_library.csv'  # Update this path / Atualize este caminho
output_dir = repo_root / 'results' / 'meta_library' / 'heatmaps' / 'notebook_example'
output_dir.mkdir(parents=True, exist_ok=True)

# Heatmap parameters / Parâmetros do mapa de calor
fields_to_plot = ['phase_TE', 'amp_TE', 'phase_TM', 'amp_TM']
grid_bins = (100, 100)  # (bins_x, bins_y)
colormap = 'viridis'

print(f'Library file: {library_file}')
print(f'Output directory: {output_dir}')

## Run / Execução

### Step 1: Load Library / Passo 1: Carregar Biblioteca

In [ ]:
# Load library / Carregar biblioteca
if library_file.suffix == '.parquet':
    df = pd.read_parquet(library_file)
else:
    df = pd.read_csv(library_file)

print(f'Loaded {len(df)} rows with {len(df.columns)} columns')
print(f'\nColumns: {df.columns.tolist()}')
print(f'\nFirst rows:')
df.head()

### Step 2: Compute Heatmaps / Passo 2: Computar Mapas de Calor

In [ ]:
# Compute heatmaps / Computar mapas de calor
heatmaps = phase_matching.compute_heatmaps(
    df,
    x='L_x',
    y='L_y',
    fields=tuple(fields_to_plot),
    bins_x=grid_bins[0],
    bins_y=grid_bins[1]
)

print('✓ Heatmaps computed / Mapas de calor computados')
print(f'Available fields: {[k for k in heatmaps.keys() if k not in ["x_grid", "y_grid", "X_grid", "Y_grid"]]}')

### Step 3: Visualize Heatmaps / Passo 3: Visualizar Mapas de Calor

In [ ]:
# Plot heatmaps / Plotar mapas de calor
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

extent = [heatmaps['x_grid'].min(), heatmaps['x_grid'].max(),
          heatmaps['y_grid'].min(), heatmaps['y_grid'].max()]

for i, field in enumerate(fields_to_plot):
    im = axes[i].imshow(heatmaps[field], origin='lower', extent=extent,
                       aspect='auto', cmap=colormap)
    axes[i].set_xlabel('L_x (nm)')
    axes[i].set_ylabel('L_y (nm)')
    axes[i].set_title(field)
    plt.colorbar(im, ax=axes[i])

plt.tight_layout()
plt.savefig(output_dir / 'heatmaps_summary.png', dpi=300, bbox_inches='tight')
plt.show()

print('✓ Heatmaps visualized and saved / Mapas visualizados e salvos')

## Results / Resultados

### Interpretation / Interpretação

**Phase Heatmaps (phase_TE, phase_TM) / Mapas de Fase:**
- Show the phase response across the L_x × L_y space
- Mostram a resposta de fase através do espaço L_x × L_y
- Continuous regions indicate smooth phase transitions
- Regiões contínuas indicam transições de fase suaves

**Amplitude Heatmaps (amp_TE, amp_TM) / Mapas de Amplitude:**
- Show transmission efficiency across parameter space
- Mostram eficiência de transmissão através do espaço de parâmetros
- Bright regions indicate high transmission
- Regiões brilhantes indicam alta transmissão

### Applications / Aplicações

1. **Parameter Space Exploration / Exploração do Espaço de Parâmetros**
   - Identify regions with desired properties
   - Identificar regiões com propriedades desejadas

2. **Phase Matching Preparation / Preparação de Casamento de Fase**
   - Understand available phase coverage
   - Entender cobertura de fase disponível

3. **Design Optimization / Otimização de Design**
   - Find optimal regions for specific targets
   - Encontrar regiões ótimas para alvos específicos

## Reproducibility / Reprodutibilidade

### Using CLI / Usando CLI

The same results can be generated using the command line:

Os mesmos resultados podem ser gerados usando a linha de comando:

```bash
python src/cli/run_heatmaps.py \
  --library library_cleaned.csv \
  --fields phase_TE amp_TE phase_TM amp_TM \
  --bins-x 100 --bins-y 100 \
  --colormap viridis \
  --experiment notebook_example
```

### Python API

```python
from meta_library import phase_matching
import pandas as pd

df = pd.read_csv('library_cleaned.csv')
heatmaps = phase_matching.compute_heatmaps(
    df, fields=('phase_TE', 'amp_TE', 'phase_TM', 'amp_TM')
)
phase_matching.save_heatmap_figures(heatmaps, out_dir='output')
```